# LongContextReorder로 문서 재정렬 최적화

## 📚 개요

LangChain의 `LongContextReorder`를 활용하여 긴 문맥에서 검색된 문서들의 순서를 재배열하는 방법을 학습합니다. 이는 모델이 긴 컨텍스트 중간에 있는 관련 정보를 놓치는 문제를 해결하는 기법입니다.

## 🎯 학습 목표

- LongContextReorder의 필요성과 원리 이해
- 문서 재정렬을 통한 성능 향상 방법 학습
- RAG 파이프라인에서의 문서 재정렬 구현

## 1. 긴 문맥 재정렬(LongContextReorder)의 필요성

### 문제점
모델의 아키텍처와 상관없이, 10개 이상의 검색된 문서를 포함할 경우 성능이 상당히 저하됩니다. 모델이 긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때, 제공된 문서를 무시하는 경향이 있습니다.

### 해결 방안
이 문제를 피하기 위해, 검색 후 문서의 순서를 재배열하여 성능 저하를 방지할 수 있습니다.
- 덜 관련된 문서는 목록의 중간에 위치
- 더 관련된 문서는 시작과 끝에 위치

### 참고 논문
- https://arxiv.org/abs/2307.03172

## 2. 환경 설정

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 3. 기본 구현: 벡터 저장소와 검색기 생성

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 테스트용 텍스트 데이터
texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

# 검색기 생성 (K=10으로 설정하여 모든 문서 검색)
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

## 4. 검색 수행 및 결과 확인

In [3]:
query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"

# 관련성 점수에 따라 정렬된 관련 문서를 가져옵니다.
docs = retriever.invoke(query)

print("원본 검색 결과 (관련성 점수 순):")
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}")

원본 검색 결과 (관련성 점수 순):
1. ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
2. ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
3. 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
4. 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
5. 이건 그냥 내가 아무렇게나 적어본 글입니다.
6. 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
7. 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
8. 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
9. 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
10. FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.


## 5. LongContextReorder를 사용한 문서 재정렬

In [4]:
# LongContextReorder 인스턴스 생성
reordering = LongContextReorder()

# 문서 재정렬
# 덜 관련된 문서는 목록의 중간에 위치하고 더 관련된 문서는 시작/끝에 위치합니다.
reordered_docs = reordering.transform_documents(docs)

print("재정렬된 문서 (관련 문서가 시작과 끝에 위치):")
for i, doc in enumerate(reordered_docs, 1):
    print(f"{i}. {doc.page_content}")

재정렬된 문서 (관련 문서가 시작과 끝에 위치):
1. ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
2. 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
3. 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
4. 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
5. FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.
6. 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
7. 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
8. 이건 그냥 내가 아무렇게나 적어본 글입니다.
9. 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
10. ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.


## 6. Context Reordering을 활용한 RAG 체인 구현

재정렬 기능을 RAG(Retrieval-Augmented Generation) 파이프라인에 통합합니다.

In [5]:
# 문서 포맷팅 함수 (기본)
def format_docs_simple(docs):
    return "\n".join([doc.page_content for doc in docs])

# 문서 포맷팅 함수 (인덱스와 소스 정보 포함)
def format_docs_with_source(docs):
    return "\n".join(
        [
            f"[{i}] {doc.page_content} [source: example@email.com]"
            for i, doc in enumerate(docs)
        ]
    )

# 문서 재정렬 및 포맷팅 함수
def reorder_and_format_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    
    # 포맷팅
    formatted = format_docs_with_source(reordered_docs)
    print("재정렬된 문서:")
    print(formatted)
    print("-" * 50)
    
    return formatted

In [6]:
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# 프롬프트 템플릿 정의
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the following language: {language}
"""

# 프롬프트 생성
prompt = ChatPromptTemplate.from_template(template)

# RAG 체인 구성
chain = (
    {
        "context": itemgetter("question")
        | retriever
        | RunnableLambda(reorder_and_format_documents),  # 문서 재정렬 적용
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

## 7. RAG 체인 실행 및 결과 확인

In [7]:
# 체인 실행
answer = chain.invoke(
    {
        "question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?", 
        "language": "Korean"
    }
)

print("\n최종 답변:")
print(answer)

재정렬된 문서:
[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: example@email.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: example@email.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: example@email.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: example@email.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: example@email.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: example@email.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: example@email.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: example@email.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: example@email.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: example@email.com]
--------------------------------------------------

최종 답변:
ChatGPT는 OpenAI에 의해 개발된 인공지능으로, 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. 지속적인 학습과 업데이트를 통해 기능이 발전하고 있으며, 다양한 질문에 답할 수 있도록 설계되었습니다. 또한, 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에

## 8. 재정렬 전후 비교

문서 재정렬이 어떻게 작동하는지 시각적으로 비교해봅시다.

In [8]:
print("=== 문서 순서 비교 ===")
print("\n원본 순서 (관련성 점수 순):")
for i, doc in enumerate(docs, 1):
    content_preview = doc.page_content[:20] + "..."
    if i == 1:
        print(f"{i}. {content_preview} (가장 관련성 높음)")
    elif i == 5:
        print(f"{i}. {content_preview} (중간)")
    elif i == 10:
        print(f"{i}. {content_preview} (가장 관련성 낮음)")
    else:
        print(f"{i}. {content_preview}")

print("\n재정렬 후 (시작과 끝에 중요 문서):")
for i, doc in enumerate(reordered_docs, 1):
    content_preview = doc.page_content[:20] + "..."
    if "ChatGPT" in doc.page_content or "챗GPT" in doc.page_content or "챗지피티" in doc.page_content:
        print(f"{i}. {content_preview} ⭐")
    elif i == 5:
        print(f"{i}. {content_preview} (중간에 배치)")
    else:
        print(f"{i}. {content_preview}")

print("\n⭐ 표시: ChatGPT 관련 중요 문서")

=== 문서 순서 비교 ===

원본 순서 (관련성 점수 순):
1. ChatGPT는 복잡한 문제를... (가장 관련성 높음)
2. ChatGPT의 기능은 지속적인...
3. 사용자와 대화하는 것처럼...
4. 챗GPT는 OpenAI에 의해...
5. 이건 그냥 내가... (중간)
6. 챗지피티는 사용자의 질문을...
7. 비트코인은 디지털 금이라고...
8. 아이폰, 아이패드, 맥북...
9. 애플 워치와 에어팟...
10. FIFA 월드컵은... (가장 관련성 낮음)

재정렬 후 (시작과 끝에 중요 문서):
1. ChatGPT의 기능은 지속적인... ⭐
2. 챗GPT는 OpenAI에 의해... ⭐
3. 챗지피티는 사용자의 질문을... ⭐
4. 아이폰, 아이패드, 맥북...
5. FIFA 월드컵은... (중간에 배치)
6. 애플 워치와 에어팟...
7. 비트코인은 디지털 금이라고...
8. 이건 그냥 내가...
9. 사용자와 대화하는 것처럼... ⭐
10. ChatGPT는 복잡한 문제를... ⭐

⭐ 표시: ChatGPT 관련 중요 문서


## 📝 요약

### 핵심 개념
1. **문제**: 긴 컨텍스트에서 중간에 있는 정보를 모델이 놓치는 경향
2. **해결책**: LongContextReorder를 사용한 문서 재정렬
3. **원리**: 관련성 높은 문서를 시작과 끝에 배치

### 구현 단계
1. 벡터 저장소 생성 및 검색기 설정
2. LongContextReorder 인스턴스 생성
3. transform_documents() 메서드로 문서 재정렬
4. RAG 체인에 RunnableLambda로 통합

### 활용 시나리오
- 10개 이상의 문서를 검색하는 경우
- 긴 컨텍스트를 다루는 RAG 시스템
- 검색 품질 향상이 필요한 경우

### 주의사항
- 검색된 문서 수가 적을 때는 효과가 제한적일 수 있음
- 재정렬 알고리즘의 특성상 추가적인 처리 시간 필요